In [0]:
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from __future__ import print_function, division
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import os
import sys
import tarfile
import numpy as np
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw
import h5py
%matplotlib inline

In [0]:
import torchvision.datasets as dsets 
import torchvision.transforms as transforms 
import torch.nn.functional as fun
from torch.autograd import Variable 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Hyper Parameters  
input_size = 784
num_classes = 26

In [3]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
cd /content/gdrive/'My Drive'/

/content/gdrive/My Drive


In [0]:
train = 'competition.csv'
test = 'test.csv'

In [0]:
from sklearn.preprocessing import normalize

In [0]:
class Train(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.samples = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        labels = self.samples.iloc[idx, 0]
        labels= labels - 1
        #labels = np.array(labels)
        pixels = self.samples.iloc[idx, 1:785]
        labels = np.array(labels)
        pixels = np.array(pixels).reshape(28,28)
        pixels = normalize(pixels)
        pixels = pixels.reshape(1,28,28)
        pixels = torch.from_numpy(pixels)
        labels = torch.from_numpy(labels)
        #sample = {}

        #if self.transform:
            #sample = self.transform(sample)
        return pixels, labels

In [0]:
class Test(Dataset):
    def __init__(self, csv_file, root_dir, transform=None):
        self.samples = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform
    def __len__(self):
        return len(self.samples)
    def __getitem__(self, idx):
        pixels = self.samples.iloc[idx, 0:784]
        pixels = np.array(pixels).reshape(28,28)
        pixels = normalize(pixels)
        pixels = pixels.reshape(1,28,28)
        pixels = torch.from_numpy(pixels)

        return pixels

In [0]:
train = Train(csv_file='competition.csv', root_dir='')#train = Train(csv_file='competition.csv', root_dir='', transform = transforms.ToTensor())

In [0]:
full_dataset = train
train_size = int(0.8 * len(full_dataset))
test_size = len(full_dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

In [0]:
train_loader = DataLoader(train_dataset,batch_size=100,
                        shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset,batch_size=1,
                        shuffle=False, num_workers=4)

In [0]:
class LeNET(nn.Module):
    def __init__(self):
        super(LeNET, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=5)
        self.conv2 = nn.Conv2d(32, 128, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(2048, 1024)
        self.fc3 = nn.Linear(1024, 512)
        self.fc5 = nn.Linear(512, 26)

    def forward(self, x): 
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        #print(x.size())
        x = x.view(-1, 2048)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = F.relu(self.fc3(x))
        x = F.dropout(x, training=self.training)
        x = self.fc5(x)
        return x 



In [0]:
model = LeNET() 
model.to(device)
criterion = nn.CrossEntropyLoss() 
#optimizer = torch.optim.Adadelta(model.parameters(), lr = 0.01)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.01) 

In [0]:
num_epochs = 100

In [98]:
correct = 0
total = 0
for pixels, labels in test_loader:
    labels = labels.cuda() 
    pixels = pixels.cuda() 
    outputs = model(pixels.float()) 
    _, predicted = torch.max(outputs.data, 1) 
    total += labels.size(0) 
    correct += (predicted == labels).sum() 
  
print('Accuracy of the model on the 10000 test images: % d %%' % ( 
            100 * correct / total))     

Accuracy of the model on the 10000 test images:  58 %


In [0]:
test = Test(csv_file='test.csv', root_dir='')#test = Train(csv_file='test.csv', root_dir='', transform = transforms.ToTensor())

In [0]:
train_loader = DataLoader(train,batch_size=100,
                        shuffle=True, num_workers=4)
test_loader = DataLoader(test,batch_size=1,
                        shuffle=False, num_workers=4)

In [0]:
len(train), len(test)

(40000, 10000)

In [0]:
for i, (pixels, labels) in enumerate(train_loader):
  print(pixels.float())
  break;

In [0]:
learning_rate = 0.0001

In [0]:
class NET(nn.Module):
    def __init__(self):
        super(NET, self).__init__() 
        self.conv1 = nn.Conv2d(1, 28, 7, 1, 0)
        self.maxpool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(6, 16, 2, 1, 1)
        self.maxpool2 = nn.MaxPool2d(2)
        self.relu = nn.ReLU() 
        self.dropout2d = nn.Dropout2d()
        self.fc1 = nn.Linear(576, 1000)
        self.fc2 = nn.Linear(1000, 26)

    def forward(self, x): 
        out = self.relu(self.maxpool1(self.conv1(x)))
        out = self.relu(self.maxpool2(self.dropout2d(self.conv2(out))))
        out = out.view(-1, 576)
        out = self.relu(self.fc1(out)) 
        out = self.fc2(out) 
        return out 



In [0]:
class MyNET(nn.Module):
    def __init__(self):
        super(MyNET, self).__init__() 
        hid1 = nn.Sequential(
            nn.Conv2d(in_channels=1, out_channels=56, kernel_size=2, padding=2),#выход28x28
            nn.BatchNorm2d(num_features=56),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            nn.Dropout(0.2)
        )#выход15x15
        hid2 = nn.Sequential(
            nn.Conv2d(in_channels=56, out_channels=64, kernel_size=2, padding=2),
            nn.BatchNorm2d(num_features=64),
            nn.ReLU(),#18x18
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            nn.Dropout(0.2) #19x19
        )
        hid3 = nn.Sequential(
            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=2, padding=2),
            nn.BatchNorm2d(num_features=128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2, padding=1),
            nn.Dropout(0.2)
        )
        hid4 = nn.Sequential(
            nn.Conv2d(in_channels=128, out_channels=160, kernel_size=2, padding=2),
            nn.BatchNorm2d(num_features=160),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
            nn.Dropout(0.2)
        )
        hid5 = nn.Sequential(
            nn.Conv2d(in_channels=160, out_channels=192, kernel_size=2, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=1, padding=1),
            nn.Dropout(0.2)
        )
        hid6 = nn.Sequential(
            nn.Conv2d(in_channels=192, out_channels=192, kernel_size=2, padding=1),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=1, stride=1, padding=0),
            nn.Dropout(0.2)
        )
        hid7 = nn.Sequential(
            nn.Conv2d(in_channels=192, out_channels=192, kernel_size=2, padding=2),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=1, stride=2, padding=0),
            nn.Dropout(0.2)
        )
        hid8 = nn.Sequential(
            nn.Conv2d(in_channels=192, out_channels=192, kernel_size=2, padding=1),
            nn.BatchNorm2d(num_features=192),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=1, stride=2, padding=0),
            nn.Dropout(0.2)
        )
        hid9 = nn.Sequential(
            nn.Linear(192 * 6 * 6, 576),
            nn.ReLU()
        )
        hid10 = nn.Sequential(
            nn.Linear(576, 26),
            nn.ReLU()
        )
        self._features = nn.Sequential(
            hid1,
            hid2,
            hid3,
            hid4,
            hid5,
            hid6,
            hid7,
            hid8
        )
        self._classifier = nn.Sequential(
            hid9,
            hid10
        )
    def forward(self, x): 
        x = self._features(x)
        x = x.view(x.size(0), 192 * 6 * 6)
        x = self._classifier(x)
        return x

In [0]:
model = MyNET() 
model.to(device)
criterion = nn.CrossEntropyLoss() 
#optimizer = torch.optim.Adadelta(model.parameters(), lr = 0.1)
optimizer = torch.optim.SGD(model.parameters(), lr = 0.1) 

In [0]:
num_epochs = 1000
optimizer = torch.optim.SGD(model.parameters(), lr = 0.0001) 

In [0]:
for epoch in range(num_epochs): 
    for i, (pixels, labels) in enumerate(train_loader): 
        
        optimizer.zero_grad() 
        pixels = pixels.cuda()
        labels = labels.cuda()
        outputs = model(pixels.float())
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
  
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, 
                     len(train) // 100, loss.item())) 
  


Epoch: [ 6/ 100], Step: [ 300/ 400], Loss: 0.5729
Epoch: [ 6/ 100], Step: [ 400/ 400], Loss: 0.4582


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f624cfa21d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f624cfa21d0>>
Traceback (most recent call last):
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x

Epoch: [ 7/ 100], Step: [ 100/ 400], Loss: 0.4120
Epoch: [ 7/ 100], Step: [ 200/ 400], Loss: 0.3959
Epoch: [ 7/ 100], Step: [ 300/ 400], Loss: 0.6023
Epoch: [ 7/ 100], Step: [ 400/ 400], Loss: 0.5962


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f624cfa21d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f624cfa21d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/usr/local/l

Epoch: [ 8/ 100], Step: [ 100/ 400], Loss: 0.5786
Epoch: [ 8/ 100], Step: [ 200/ 400], Loss: 0.3486
Epoch: [ 8/ 100], Step: [ 300/ 400], Loss: 0.4056
Epoch: [ 8/ 100], Step: [ 400/ 400], Loss: 0.5741


...

Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f624cfa21d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f624cfa21d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
    self._shutdown_workers()
  File "/usr/local/l

Epoch: [ 17/ 100], Step: [ 100/ 400], Loss: 0.2224
Epoch: [ 17/ 100], Step: [ 200/ 400], Loss: 0.4039
Epoch: [ 17/ 100], Step: [ 300/ 400], Loss: 0.4338
Epoch: [ 17/ 100], Step: [ 400/ 400], Loss: 0.4841


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f624cfa21d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f624cfa21d0>>
Traceback (most recent call last):
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f624cfa21d0>>
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/pr

Epoch: [ 18/ 100], Step: [ 100/ 400], Loss: 0.4889
Epoch: [ 18/ 100], Step: [ 200/ 400], Loss: 0.5089
Epoch: [ 18/ 100], Step: [ 300/ 400], Loss: 0.5406
Epoch: [ 18/ 100], Step: [ 400/ 400], Loss: 0.4222


Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f624cfa21d0>>
Traceback (most recent call last):
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 926, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.6/dist-packages/torch/utils/data/dataloader.py", line 906, in _shutdown_workers
    w.join()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 122, in join
    assert self._parent_pid == os.getpid(), 'can only join a child process'
AssertionError: can only join a child process
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x7f624cfa21d0>>
Traceback (most recent call last):
Exception ignored in: <bound method _MultiProcessingDataLoaderIter.__del__ of <torch.utils.data.dataloader._MultiProcessingDataLoaderIter object at 0x

In [0]:
for epoch in range(num_epochs): 
    for i, (pixels, labels) in enumerate(train_loader): 
        
        optimizer.zero_grad() 
        pixels = pixels.cuda()
        labels = labels.cuda()
        outputs = model(pixels.float())
        loss = criterion(outputs, labels) 
        loss.backward() 
        optimizer.step() 
  
        if (i + 1) % 100 == 0: 
            print('Epoch: [% d/% d], Step: [% d/% d], Loss: %.4f'
                  % (epoch + 1, num_epochs, i + 1, 
                     len(train) // 100, loss.item())) 

In [0]:
outputs = []
for pixels in test_loader: 
    pixels = pixels.cuda()
    #pixels = torch.stack(pixels).to(device)
    predicted = model(pixels.float())
    #predicted = predicted.cpu().numpy()
    _, predicted = torch.max(predicted.data, 1) 
    predicted = predicted.cpu().numpy()
    outputs.append(predicted)

In [0]:
len(outputs)

10000

In [0]:
out = []
for i in outputs:
  out.append(i[0]+1)

In [0]:
submission = pd.DataFrame(columns=['Id', 'Predicted'])
submission['Id'] = list(range(10000))
submission['Predicted'] = out[0:]

In [0]:
submission.to_csv('mypred6.csv', index=False)

In [0]:
model

In [0]:
submission

,Id,Predicted
0,0,5
1,1,24
2,2,9
3,3,16
4,4,22
...,...,...
9995,9995,20
9996,9996,11
9997,9997,8
9998,9998,26


In [0]:
ls

In [0]:
torch.save(model.state_dict(), 'Sorevmodelgpu.pth')